In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import os
import json
import folium
from folium import plugins
import re
from haversine import haversine, Unit
from datetime import datetime, date, time
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from collections import namedtuple
import geopandas as gpd
from folium.plugins import HeatMap

### Data from Bixi

In [2]:
data = pd.read_pickle("paths")

In [3]:
data

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
0,4000,4000,103,1173.126214,45.549598,-73.541874,45.549598,-73.541874,0.000000
1,4000,4002,1,1864.000000,45.549598,-73.541874,45.516937,-73.640483,8.496129
2,4000,5007,1,3585.000000,45.549598,-73.541874,45.523319,-73.520127,3.377533
3,4000,6001,53,1411.509434,45.549598,-73.541874,45.510351,-73.556508,4.510499
4,4000,6002,95,504.852632,45.549598,-73.541874,45.539385,-73.541000,1.137631
...,...,...,...,...,...,...,...,...,...
220221,10002,7143,55,1530.654545,45.478228,-73.569651,45.446934,-73.572199,3.485341
220222,10002,7144,137,594.781022,45.478228,-73.569651,45.464893,-73.566998,1.497191
220223,10002,7145,25,993.280000,45.478228,-73.569651,45.453319,-73.576775,2.824937
220224,10002,7146,1,689.000000,45.478228,-73.569651,45.492222,-73.569348,1.556301


In [4]:
data = data.sort_values(by="counts", axis=0)

In [5]:
data

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
105684,6254,7100,1,1693.000000,45.540060,-73.608970,45.494044,-73.654495,6.225930
32904,6084,7018,1,1819.000000,45.495581,-73.553711,45.545230,-73.537250,5.667654
127414,6339,6047,1,1984.000000,45.474565,-73.624036,45.507402,-73.578444,5.095292
32907,6084,7030,1,1827.000000,45.495581,-73.553711,45.533409,-73.570657,4.408642
32908,6084,7031,1,1355.000000,45.495581,-73.553711,45.522878,-73.577495,3.556386
...,...,...,...,...,...,...,...,...,...
52492,6136,6163,2149,333.435551,45.527616,-73.589115,45.535430,-73.582200,1.022253
15127,6036,6036,2440,2081.688525,45.504242,-73.553469,45.504242,-73.553469,0.000000
21908,6052,6406,2445,1379.795092,45.497515,-73.552571,45.480208,-73.577599,2.740398
11876,6026,6026,2871,1850.238245,45.507610,-73.551836,45.507610,-73.551836,0.000000


In [6]:
data = data.dropna()

In [7]:
data = data[data['distance']>0.]

In [8]:
data

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
105684,6254,7100,1,1693.000000,45.540060,-73.608970,45.494044,-73.654495,6.225930
32904,6084,7018,1,1819.000000,45.495581,-73.553711,45.545230,-73.537250,5.667654
127414,6339,6047,1,1984.000000,45.474565,-73.624036,45.507402,-73.578444,5.095292
32907,6084,7030,1,1827.000000,45.495581,-73.553711,45.533409,-73.570657,4.408642
32908,6084,7031,1,1355.000000,45.495581,-73.553711,45.522878,-73.577495,3.556386
...,...,...,...,...,...,...,...,...,...
52494,6136,6165,2059,237.354541,45.527616,-73.589115,45.532514,-73.584811,0.639508
144330,6396,6398,2090,267.561722,45.554214,-73.551560,45.551192,-73.540872,0.897440
21623,6052,6026,2128,882.905545,45.497515,-73.552571,45.507610,-73.551836,1.123981
52492,6136,6163,2149,333.435551,45.527616,-73.589115,45.535430,-73.582200,1.022253


In [9]:
data[(data['counts']>=20)&(data['counts']<=30)]

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
193658,7036,6188,20,1101.900000,45.519895,-73.596827,45.526027,-73.577859,1.627478
76886,6190,6217,20,473.700000,45.527009,-73.585766,45.518967,-73.583616,0.909792
37049,6095,6748,20,833.900000,45.491800,-73.584006,45.492825,-73.557978,2.032050
127392,6339,6009,20,1534.050000,45.474565,-73.624036,45.498112,-73.577615,4.466621
52910,6137,6037,20,1399.350000,45.530347,-73.566988,45.501967,-73.555153,3.287714
...,...,...,...,...,...,...,...,...,...
173983,6908,6259,30,688.600000,45.533410,-73.601124,45.540881,-73.584157,1.560916
67426,6169,6386,30,1119.700000,45.527432,-73.579917,45.541780,-73.555714,2.469632
2516,6003,6184,30,1007.633333,45.511007,-73.567602,45.524673,-73.582550,1.914591
96795,6231,6711,30,751.700000,45.523026,-73.601840,45.531938,-73.617842,1.592394


In [35]:
data.sample(150)

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
182445,7005,7096,6,2106.166667,45.567352,-73.653787,45.538308,-73.654884,3.230724
119195,6310,6372,16,1885.312500,45.505403,-73.621166,45.545200,-73.576451,5.631728
88116,6214,6004,32,838.406250,45.516091,-73.570129,45.509655,-73.554009,1.445619
118360,6307,6148,50,680.780000,45.520178,-73.579424,45.527201,-73.564729,1.385710
143606,6394,6905,18,1066.833333,45.546976,-73.543226,45.544377,-73.581018,2.957171
...,...,...,...,...,...,...,...,...,...
38752,6099,7071,13,1240.230769,45.515065,-73.559230,45.495789,-73.582596,2.812453
843,4002,6338,5,1128.400000,45.516937,-73.640483,45.545955,-73.626992,3.393513
37232,6096,6095,12,884.916667,45.493569,-73.568576,45.491800,-73.584006,1.218769
131033,6356,6031,1,2783.000000,45.477107,-73.621438,45.504263,-73.571523,4.925144


### Data from the bike paths

In [11]:
fname = "./BikePaths/reseau_cyclable_2018_c.geojson"
bp = gpd.read_file(fname)
bp = bp[bp['Ville_MTL']=='OUI']
bp = bp.drop(columns=['ID', 'ID_TRC_GEO', 'TYPE_VOIE', 'TYPE_VOIE2', 'SEPARATEUR', 'SAISONS4', 'PROTEGE_4S', 'Ville_MTL'])
bp = bp.reset_index(drop=True)

In [12]:
bp['distances'] = bp.apply(lambda x: np.array([]),axis=1)
for k in range(len(bp)):
    distances = []
    for l in range(len(bp['geometry'][k])):
        subdistances = np.array([])
        for m in range(len(bp['geometry'][k][l].xy[0])-1):
            #distances[l] = np.append(distances[l], haversine((x.latitude_start,x.longitude_start),(x.latitude_end,x.longitude_end)))
            subdistances = np.append(subdistances, 1000*haversine((bp['geometry'][k][l].xy[1][m], bp['geometry'][k][l].xy[0][m]),(bp['geometry'][k][l].xy[1][m+1], bp['geometry'][k][l].xy[0][m+1])))
        distances.append(subdistances)
    bp['distances'][k] = np.array(distances)

C:\Users\quent\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


### Useful functions

In [13]:
def isincluded(x, y, a, b, da, db): # Attention ! da and db must be positive
    if abs(2*x - 2*a - da) <= da:
        if abs(2*y - 2*b - db) <= db:
            return 1
    return 0

In [14]:
def calculate_truncated_distance(x1, y1, x2, y2, a, b, da, db, d0, d1): # Attention ! da and db must be positive
    # Case where the first point is in the square
    if ((d0 == 1) and (d1 ==0)):
        if x1==x2:
            if y2<b:
                xt, yt = x1, b
            else:
                xt, yt = x1, b + db
        elif y1==y2:
            if x2<a:
                xt, yt = a, y1
            else:
                xt, yt = a + da, y1
        else:
            # Case y2 < b (under the square)
            if y2 < b:
                xtest = (x2-x1)*(b-y1)/(y2-y1) + x1
                if xtest < a:
                    xt, yt = a, (y2-y1)*(a-x1)/(x2-x1) + y1
                elif xtest > a + da:
                    xt, yt = a+da, (y2-y1)*(a+da-x1)/(x2-x1) + y1
                else : 
                    xt, yt = xtest, b
            # Case y2 > b + db (above the square)
            elif y2 > b+db:
                xtest = (x2-x1)*(b+db-y1)/(y2-y1) + x1
                if xtest < a:
                    xt, yt = a, (y2-y1)*(a-x1)/(x2-x1) + y1
                elif xtest > a + da:
                    xt, yt = a+da, (y2-y1)*(a+da-x1)/(x2-x1) + y1
                else : 
                    xt, yt = xtest, b+db
            # Case y in the range(b, b+db)
            else:
                if x2 <a:
                    xt, yt = a, (y2-y1)*(a-x1)/(x2-x1) + y1
                else: # Case x2>a+da
                    xt, yt = a+da, (y2-y1)*(a+da-x1)/(x2-x1) + y1
        return  1000*haversine((y1,x1),(yt,xt))
    elif ((d0 == 0) and (d1 ==1)):
        return calculate_truncated_distance(x2, y2, x1, y1, a, b, da, db, d1, d0)
    else:
        return(0.)

In [15]:
def compute_density_in_squares(xp1,yp1,xp2,yp2):
    '''(x1,y1) represents the point at the bottom left and (x2,y2) the point at the top right
    The n_x_squares number represents the number of squares between x1 and x2. 
    The length of a square according to x will then be used to calculate the number of squares according to y. '''
    
    # The point (x1, y1) must correspond to the point at the bottom left of the square.
    if xp1<=xp2:
        x1, x2 = xp1, xp2
    else:
        x1, x2 = xp2, xp1
    if yp1<=yp2:
        y1, y2 = yp1, yp2
    else:
        y1, y2 = yp2, yp1
    # We look at whether the longest length of the square is according to x or y.
    if x2-x1>y2-y1:
        y1, y2 = (y1+y2)/2 - (x2-x1)/2 , (y1+y2)/2 + (x2-x1)/2
    else:
        x1, x2 = (x1+x2)/2 - (y2-y1)/2 , (x1+x2)/2 + (y2-y1)/2
    # Convert the coordinates to a and b
    a, b = x1, y1
    da, db = x2-x1, y2-y1 # da = db
    # Calculate the total distance in this square
    total_distance_in_the_square = 0.
    for k in range(len(bp['geometry'])):
        for l in range(len(bp['geometry'][k])): # l et k sont les id d'un chemin simple
            for i in range(len(bp['geometry'][k][l].xy[0])-1): # i est l'id du point sur ce chemin
                x1, y1, x2, y2 = bp['geometry'][k][l].xy[0][i], bp['geometry'][k][l].xy[1][i] , bp['geometry'][k][l].xy[0][i+1], bp['geometry'][k][l].xy[1][i+1]
                d0, d1 = isincluded(x1, y1, a, b, da, db) , isincluded(x2, y2, a, b, da, db)
                if d0+d1==2: # Si les deux points sont dans le carré, on ajoute le premier
                    total_distance_in_the_square += bp['distances'][k][l][i]
                elif ((d1==1) or (d0==1)): # Si seul le deuxième point est dans le carré, on ajoute l'intersection seulement
                    total_distance_in_the_square += calculate_truncated_distance(x1, y1, x2, y2, a, b, da, db, d0, d1)
    total_distance_in_the_square = (total_distance_in_the_square*1e-3)/((haversine((b, a), (b+db, a)))**2) # unit : km / (km**2)
    return total_distance_in_the_square

In [16]:
subdata = data.sample(1)
subdata = subdata.reset_index(drop=True)

In [17]:
subdata['bp_density'] = bp.apply(lambda x: 0.,axis=1)

In [18]:
subdata

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance,bp_density
0,6115,7007,1,1739.000000,45.466914,-73.631704,45.512933,-73.633890,5.119922,0.0
1,6177,6015,317,423.867508,45.522426,-73.574184,45.515299,-73.561273,1.280574,0.0
2,6742,6423,1,2262.000000,45.463397,-73.596436,45.496302,-73.629822,4.490278,0.0


In [19]:
import time
start_time = time.time()
for k in subdata.index:
    subdata['bp_density'][k] = compute_density_in_squares(subdata['longitude_start'][k],subdata['latitude_start'][k],subdata['longitude_end'][k],subdata['latitude_end'][k])
elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

C:\Users\quent\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


'00:00:24'

In [20]:
subdata

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance,bp_density
0,6115,7007,1,1739.000000,45.466914,-73.631704,45.512933,-73.633890,5.119922,0.921996
1,6177,6015,317,423.867508,45.522426,-73.574184,45.515299,-73.561273,1.280574,1.786120
2,6742,6423,1,2262.000000,45.463397,-73.596436,45.496302,-73.629822,4.490278,1.012934


In [21]:
number_of_values = data.apply(lambda x: x.count(), axis=0)

In [22]:
number_of_values


start_station_code    219609
end_station_code      219609
counts                219609
duration              219609
latitude_start        219609
longitude_start       219609
latitude_end          219609
longitude_end         219609
distance              219609
dtype: int64

In [23]:
len(data[(data['counts']>=0)&(data['counts']<10)])

126506

In [24]:
len(data[(data['counts']>=10)&(data['counts']<20)])

34068

In [25]:
len(data[(data['counts']>=20)&(data['counts']<40)])

27684

In [26]:
len(data[(data['counts']>=40)&(data['counts']<100)])

22360

In [27]:
len(data[(data['counts']>=100)&(data['counts']<500)])

8546

In [28]:
len(data[data['counts']>=500])

445

In [29]:
data[(data['counts']>=10)&(data['counts']<20)].sample(3)

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
198072,7046,6427,16,1278.750000,45.502386,-73.573338,45.470696,-73.565422,3.577354
101937,6247,6033,14,873.571429,45.532924,-73.610716,45.558281,-73.583159,3.543349
176257,6916,6118,15,928.733333,45.544513,-73.571564,45.525048,-73.560036,2.343333


In [39]:
bixi_subdata = data[(data['counts']>=500)].sample(200)

cut_counts_list = [0, 10, 20, 40, 100, 500]
for k in range(len(cut_counts_list)-1):
    bixi_subdata = pd.concat([bixi_subdata, data[(data['counts']>=cut_counts_list[k])&(data['counts']<cut_counts_list[k+1])].sample(200)])

    

In [40]:
bixi_subdata

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
141302,6387,4000,540,367.100000,45.546907,-73.551557,45.549598,-73.541874,0.811189
162751,6727,10002,1015,398.568473,45.484081,-73.560918,45.478228,-73.569651,0.941929
52527,6136,6199,645,355.582946,45.527616,-73.589115,45.525931,-73.598839,0.780373
220150,10002,6732,1079,420.763670,45.478228,-73.569651,45.477923,-73.559038,0.828188
32661,6084,6043,639,377.895149,45.495581,-73.553711,45.502060,-73.562950,1.018576
...,...,...,...,...,...,...,...,...,...
44631,6115,6263,162,324.783951,45.466914,-73.631704,45.471179,-73.627002,0.599462
205067,7070,7047,148,534.952703,45.479930,-73.555921,45.493909,-73.559507,1.579284
74187,6184,6148,137,698.284672,45.524673,-73.582550,45.527201,-73.564729,1.416421
18238,6043,6036,362,765.817680,45.502060,-73.562950,45.504242,-73.553469,0.777682


In [41]:
bixi_subdata = bixi_subdata.reset_index(drop=True)

In [42]:
bixi_subdata['bp_density'] = bixi_subdata.apply(lambda x: 0.,axis=1)

In [43]:
bixi_subdata

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance,bp_density
0,6387,4000,540,367.100000,45.546907,-73.551557,45.549598,-73.541874,0.811189,0.0
1,6727,10002,1015,398.568473,45.484081,-73.560918,45.478228,-73.569651,0.941929,0.0
2,6136,6199,645,355.582946,45.527616,-73.589115,45.525931,-73.598839,0.780373,0.0
3,10002,6732,1079,420.763670,45.478228,-73.569651,45.477923,-73.559038,0.828188,0.0
4,6084,6043,639,377.895149,45.495581,-73.553711,45.502060,-73.562950,1.018576,0.0
...,...,...,...,...,...,...,...,...,...,...
1195,6115,6263,162,324.783951,45.466914,-73.631704,45.471179,-73.627002,0.599462,0.0
1196,7070,7047,148,534.952703,45.479930,-73.555921,45.493909,-73.559507,1.579284,0.0
1197,6184,6148,137,698.284672,45.524673,-73.582550,45.527201,-73.564729,1.416421,0.0
1198,6043,6036,362,765.817680,45.502060,-73.562950,45.504242,-73.553469,0.777682,0.0


In [45]:
#bixi_subdata[0:300].to_pickle("./Guillaume_Compute_Path_Density_v0")

In [46]:
#bixi_subdata[300:600].to_pickle("./Kiara_Compute_Path_Density_v0")

In [47]:
#bixi_subdata[600:900].to_pickle("./Ariane_Compute_Path_Density_v0")

In [48]:
#bixi_subdata[900:1200].to_pickle("./Quentin_Compute_Path_Density_v0")

In [ ]:
name_user = "Enter your name here"
bixi_subdata  = pd.read_pickle("./"+name_user+"_Compute_Path_Density_v0")

In [98]:
import time
start_time = time.time()
for k in bixi_subdata.index:
    bixi_subdata['bp_density'][k] = compute_density_in_squares(bixi_subdata['longitude_start'][k],bixi_subdata['latitude_start'][k],bixi_subdata['longitude_end'][k],bixi_subdata['latitude_end'][k])
elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

C:\Users\quent\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


'06:07:58'

In [99]:
bixi_subdata

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance,bp_density
0,6136,6163,2089,334.380086,45.527616,-73.589115,45.535430,-73.582200,1.022253,3.467586
1,6052,6026,2080,887.340385,45.497515,-73.552571,45.507610,-73.551836,1.123981,1.337192
2,6052,6406,2385,1380.174843,45.497515,-73.552571,45.480208,-73.577599,2.740398,2.362488
3,7116,6730,1,1733.000000,45.596796,-73.535179,45.570081,-73.573047,4.184379,1.301654
4,6031,7099,1,2032.000000,45.504263,-73.571523,45.497047,-73.643216,5.644820,1.659301
...,...,...,...,...,...,...,...,...,...,...
68,6114,6110,1163,308.029235,45.523530,-73.551990,45.529033,-73.546348,0.753393,2.821029
69,6052,6036,1256,681.863057,45.497515,-73.552571,45.504242,-73.553469,0.751328,1.948059
70,6136,7026,1061,179.029218,45.527616,-73.589115,45.527041,-73.593471,0.345296,1.361192
71,6154,6187,1340,307.902239,45.532290,-73.575440,45.525628,-73.581775,0.890077,3.753846


In [100]:
#bixi_subdata.to_pickle("./Bixi_Paths_with_Path_Density")

In [ ]:
bixi_subdata.to_pickle("./"+name_user+"_Compute_Path_Density_v1")